In [24]:
import uproot4
import awkward
import numpy as np
from uproot_methods import TLorentzVectorArray

from coffea.processor import LazyDataFrame
from coffea.analysis_objects import JaggedCandidateArray


In [25]:
fn = '/hadoop/cms/store/user/legianni/ProjectMetis/HHggtautau_Era2018_private_prova_vera/test_nanoaodSkim_3.root'
file = uproot4.open(fn)

In [26]:
tree = file['Events']
df = LazyDataFrame(tree, flatten=False)

#tree.keys()

In [27]:
# create photon candidates
photons = JaggedCandidateArray.candidatesfromcounts(
    df['nPhoton'],
    pt=df['Photon_pt'].content,
    eta=df['Photon_eta'].content,
    phi=df['Photon_phi'].content,
    mass=df['Photon_mass'].content,
)

In [28]:
# check if photons and the index array have equal length (that should always be the case, but just to be sure)
len(photons) == len(df['gHidx'])

True

In [29]:
# check the indices
df['gHidx']

<JaggedArray [[0 2] [0 2] [0 1] ... [-1 -1] [-1 -1] [0 1]] at 0x7f99733baa58>

In [30]:
# some of the indices are -1 -> this will be problematic. let's throw them away
df['gHidx'][(df['gHidx']>=0)]

<JaggedArray [[0 2] [0 2] [0 1] ... [] [] [0 1]] at 0x7f99733b2cc0>

In [31]:
# look at some of the photon pts so that we can be sure we're really selecting the right photons in the end
photons.pt

<JaggedArray [[223.92703 137.99004 110.93973 90.21473 101.65603 88.09794] [121.273 59.024254 25.527027] [158.28708 151.59003 62.73162] ... [53.33714 30.271727 18.597582 15.280681] [93.65514 48.216213 35.191925] [68.64432 53.799282 46.529034 23.732985]] at 0x7f99782e6dd8>

In [32]:
# as expected, using the indices that are not cleaned from -1 values selects some photons that shouldn't be there,
# e.g. in the second to last event
photons[df['gHidx']].pt

<JaggedArray [[223.92703 110.93973] [121.273 25.527027] [158.28708 151.59003] ... [15.280681 15.280681] [35.191925 35.191925] [68.64432 53.799282]] at 0x7f99733a6e80>

In [33]:
# this is fixed when we just use indices that are greater or equal to 0
photons[df['gHidx'][(df['gHidx']>=0)]].pt

<JaggedArray [[223.92703 110.93973] [121.273 25.527027] [158.28708 151.59003] ... [] [] [68.64432 53.799282]] at 0x7f99733a6f60>